In [2]:
# Import API modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import geopandas as gpd
# Import linestrings
import tradeAreasPreprocessing as tap
from shapely.geometry import LineString
import census_finder

In [3]:
# Read token.txt
with open("inrix_token.txt", "r") as f:
    token = f.read()
    
# Initialize class
tap = tap.tradeAreasPreprocessing(token=token,
                                  od = 'origin', 
                                  geoFilterType = 'polygon',
                                radius = '0.2km',
                                points = '37.734622%7C-122.471603%2C37.743627%7C-122.463850%2C37.743066%7C-122.475429',
                                providerType = 'consumer',
                                startDateTime = '%3E%3D2023-06-01T02%3A31',
                                endDateTime = '%3C%3D2023-06-15T02%3A31')

# Read data
df = tap.read_data()

In [4]:
df = tap.create_dataframe(df)

In [5]:
df = tap.separate_coordinates(df)

df = tap.add_temporal_vars(df)

#df = tap.add_mode(df)

In [6]:
trips_gdf = tap.get_trip_geometry(df)

ca_tiger = tap.downloadShapefile()

intersected_trips = tap.intersectedTrips(trips_gdf, ca_tiger)

Using FIPS code '06' for input 'CA'


In [7]:
# read in files
geojson_location = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/sf-tracts-2020-clipped.geojson"
census_data = "/Users/ekinokos2/Library/CloudStorage/OneDrive-UW/GeoAdVisor/census_data/census_data_clean.csv"

# Load the GeoJSON file into a GeoDataFrame and csv into df
gdf = gpd.read_file(geojson_location)
census_df = pd.read_csv(census_data, dtype={"tract": str})

# Filter out invalid geometries
gdf = gdf[gdf.is_valid]

# INitialize class
cf = census_finder.censusFinder(gdf, census_df)

tract_no = cf.get_census_tract_info(37.734622, -122.471603)

demographics = cf.lookup_demographics(tract_no)

In [8]:
def append_demographics(row):
    tract_no = cf.get_census_tract_info(row['start_lat'], row['start_lon'])
    demographics = cf.lookup_demographics(tract_no)
    return demographics.iloc[0]

demographics_df = df.apply(append_demographics, axis=1)

result_df = pd.concat([df, demographics_df], axis=1).iloc[:, :37]

In [9]:
# Merge result_df and intersected trips on tripId
intersected_trips_demo = pd.merge(result_df, intersected_trips, on='tripId')

In [10]:
intersected_trips_demo

,providerHash,startLoc,startDateTime,endLoc,endpointType,tripDistanceMeters,tripMeanSpeedKPH,tripId,endDateTime,deviceId,...,pct_children,pct_20-39,pct_39-64,pct_65+,pct_nonwhite,pct_latine,GEOID,geometry_x,geometry_y,line_locs
0,c3e878e27f52e2a57ace4d9a76fd9acf,"37.743138,-122.466007",2023-06-01 02:35:57.079,"37.735084,-122.500487",0,3472,35,dd8ed7439a33ebe0797e23cb7918ec21,2023-06-01 02:41:56.818,c05484db3c0e831f242bdc0ce0c5e847,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075030800,"POLYGON ((-122.47573 37.74305, -122.47558 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(0.0, 1094.7372454378553)"
1,c3e878e27f52e2a57ace4d9a76fd9acf,"37.743138,-122.466007",2023-06-01 02:35:57.079,"37.735084,-122.500487",0,3472,35,dd8ed7439a33ebe0797e23cb7918ec21,2023-06-01 02:41:56.818,c05484db3c0e831f242bdc0ce0c5e847,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075033001,"POLYGON ((-122.49451 37.74222, -122.49436 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(2878.403871091762, 3208.2974503973883)"
2,c3e878e27f52e2a57ace4d9a76fd9acf,"37.743138,-122.466007",2023-06-01 02:35:57.079,"37.735084,-122.500487",0,3472,35,dd8ed7439a33ebe0797e23cb7918ec21,2023-06-01 02:41:56.818,c05484db3c0e831f242bdc0ce0c5e847,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075033002,"POLYGON ((-122.49129 37.73730, -122.49090 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(1094.7372454378553, 2878.403871091762)"
3,c3e878e27f52e2a57ace4d9a76fd9acf,"37.743138,-122.466007",2023-06-01 02:35:57.079,"37.735084,-122.500487",0,3472,35,dd8ed7439a33ebe0797e23cb7918ec21,2023-06-01 02:41:56.818,c05484db3c0e831f242bdc0ce0c5e847,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075060400,"POLYGON ((-122.51273 37.73509, -122.51008 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(3931.2131256589555, 3934.2353148580683)"
4,c3e878e27f52e2a57ace4d9a76fd9acf,"37.743138,-122.466007",2023-06-01 02:35:57.079,"37.735084,-122.500487",0,3472,35,dd8ed7439a33ebe0797e23cb7918ec21,2023-06-01 02:41:56.818,c05484db3c0e831f242bdc0ce0c5e847,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075035300,"POLYGON ((-122.50193 37.74938, -122.50086 37.7...","LINESTRING (-122.46601 37.74314, -122.50049 37...","(3208.2974503973883, 3931.2131256589555)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4217,c3e878e27f52e2a57ace4d9a76fd9acf,"37.738366,-122.47101",2023-06-15 02:03:58.317,"37.73351,-122.43355",3,3609,18,485245869854ea94394727d7633ce978,2023-06-15 02:15:56.648,b517f0d628f83c6f9e16d4ce57273da1,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075030800,"POLYGON ((-122.47573 37.74305, -122.47558 37.7...","LINESTRING (-122.47101 37.73837, -122.43355 37...","(0.0, 1184.701405072554)"
4218,c3e878e27f52e2a57ace4d9a76fd9acf,"37.738366,-122.47101",2023-06-15 02:03:58.317,"37.73351,-122.43355",3,3609,18,485245869854ea94394727d7633ce978,2023-06-15 02:15:56.648,b517f0d628f83c6f9e16d4ce57273da1,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075031100,"POLYGON ((-122.45407 37.72984, -122.45340 37.7...","LINESTRING (-122.47101 37.73837, -122.43355 37...","(3645.571043805868, 3986.8768432200436)"
4219,c3e878e27f52e2a57ace4d9a76fd9acf,"37.738366,-122.47101",2023-06-15 02:03:58.317,"37.73351,-122.43355",3,3609,18,485245869854ea94394727d7633ce978,2023-06-15 02:15:56.648,b517f0d628f83c6f9e16d4ce57273da1,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075030700,"POLYGON ((-122.46043 37.73766, -122.46038 37.7...","LINESTRING (-122.47101 37.73837, -122.43355 37...","(1184.701405072554, 3645.571043805868)"
4220,c3e878e27f52e2a57ace4d9a76fd9acf,"37.738366,-122.47101",2023-06-15 02:03:58.317,"37.73351,-122.43355",3,3609,18,485245869854ea94394727d7633ce978,2023-06-15 02:15:56.648,b517f0d628f83c6f9e16d4ce57273da1,...,0.10048,0.023506,0.190697,0.098659,0.338851,0.049164,06075021800,"POLYGON ((-122.43584 37.73388, -122.43583 37.7...","LINESTR